In [1]:
# importing major libraries 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import warnings 
warnings.filterwarnings('ignore')

In [2]:
# pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = pd.read_csv('train_titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
# we have null values in Age and Cabin

In [9]:
# plan/pipeline

# missing values -> ohe -> Scaling -> feature selection -> train model 

In [12]:
df.columns
df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

In [15]:
x= df.iloc[:,1:]
y= df.Survived

In [16]:
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.2,random_state=42)

In [20]:
print('rows for training\t',x_train.shape[0])
print('rows for testing\t',x_test.shape[0])

rows for training	 712
rows for testing	 179


In [32]:
# imputer 

trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])],
     remainder='passthrough')

In [33]:
trf1

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [34]:
# one hot encoding 

trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[1,6])],
    remainder= 'passthrough'
)
trf2

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_embarked',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 [1, 6])])

In [40]:
# scaling 

trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10)) 
])
trf3

ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))])

In [43]:
trf4 = SelectKBest(score_func=chi2,k=8)
trf4

SelectKBest(k=8, score_func=<function chi2 at 0x000001DD71FCDEE0>)

In [44]:
trf5 = DecisionTreeClassifier()
trf5

DecisionTreeClassifier()

In [81]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)]
)
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001DD71FCDEE0>)),
                ('trf5', DecisionTreeClassifier())])

In [82]:
pipe1 = make_pipeline(trf1,trf2,trf3,trf4,trf5)
pipe1

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001DD71FCDEE0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [83]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001DD71FCDEE0>)),
                ('trf5', DecisionTreeClassifier())])

In [84]:
pipe.named_steps 

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x000001DD71FCDEE0>),
 'trf5': DecisionTreeClassifier()}

In [85]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [86]:
y_pred = pipe.predict(x_test)

In [87]:
accuracy_score(y_test,y_pred)

0.6256983240223464

In [88]:
# grid_seacrch cv 

In [94]:
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [95]:
from sklearn.model_selection import GridSearchCV

In [101]:
grid = GridSearchCV(pipe,params,cv=5, scoring='accuracy')
grid.fit(x_train,y_train)
grid.best_params_
# grid.best_score

{'trf5__max_depth': 2}

In [103]:
grid.best_score_

0.6391214419383433

In [105]:
# exporting pipeline

import pickle 

pickle.dump(pipe,open('pipe.pkl','wb'))